# ADIOS2-C++11-MPI: Write Read Cooling Rod 1D

## Writer: Generate Data
We use a simple solution to the heat transfer equation in a (very-fast) 1D cooling rod.

In [1]:
#include <cmath> //std::sin, std::exp
#include <cstddef> //std::size_t

#include <exception>
#include <iostream> //std::cout
#include <vector>
#include <string>

//this is a cling thing only
#pragma cling add_include_path("/home/wgodoy/anaconda3/envs/cling-openmpi/include")
#pragma cling add_include_path("/home/wgodoy/anaconda3/envs/cling-openmpi/include/adios2")
#pragma cling add_library_path("/home/wgodoy/anaconda3/envs/cling-openmpi/lib/")

#pragma cling load("mpi.h")
#pragma cling load("mpi_cxx")
#pragma cling load("adios2")

#include <mpi.h>
#include <adios2.h>

constexpr double pi = 3.14159265358979323846;


In [2]:
/**
 * Calculate U(Uo,x,t) -> f(initial value, location, time)
 */
double calcUxt(const double Uo, const double x, const double t)
{
    return 4 * Uo / pi * std::sin( pi * x ) * std::exp( - pi * pi * t);
}

In xeus-cling we need a lambda function, typical C++ code will just use `main`

In [3]:
// int main(int argc, char* argv[] )
[](){
auto try_and_err = []() {

    
    MPI_Init(NULL, NULL);
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    try{
        
    
        constexpr std::size_t Nx = 10;
        const adios2::Dims shape{static_cast<std::size_t>(Nx * size)};
        const adios2::Dims start{static_cast<std::size_t>(Nx * rank)};
        const adios2::Dims count{Nx};
        
        //Initialize problem x coord as cell centered
        constexpr   double Lx = 1.;
        const       double dx = Lx/shape[0];
        
        // data x
        std::vector<double> x(Nx,0);
        for(auto i=0;i < Nx; ++i)
        {
            x[i] = start[0] + (i+0.5) * dx;
        }
        
        // data U
        constexpr double Uo = 1.;
        std::vector<double> Uxt(Nx,Uo);
        
        //Initialize ADIOS2
        adios2::ADIOS adios(MPI_COMM_WORLD, adios2::DebugON);
        adios2::IO io = adios.DeclareIO("CoolingRod1D");
        
        // Define some variables
        adios2::Variable<double> var_x = 
            io.DefineVariable<double>("x", shape, start, count, adios2::ConstantDims);
        adios2::Variable<double> var_Uxt = 
            io.DefineVariable<double>("Uxt", shape, start, count, adios2::ConstantDims);
        
        adios2::Engine fw = io.Open("Uxt.bp", adios2::Mode::Write);
        
        for(auto t=0; t < 5; ++t )
        {
            //natural API for "stepping" variables, user doesn't need to track "time"
            fw.BeginStep();
            
            //We save X only once at t == 0 
            if(t==0)
            {
                fw.Put(var_x, x.data() ); //deferred mode, not saving here, but "registering"
            }
            
            //computation
            for(auto i = 0; i < Nx; ++i )
            {
                Uxt[i] = calcUxt(Uo, x[i], t);
            }
            
            // I/O
            fw.Put(var_Uxt, Uxt.data() );
            
            // x and Uxt are "consumed"/"collected" at EndStep
            fw.EndStep();
        }
        // Close in C++ is optional as it's called by the adios destructor (RAII)
        fw.Close();
    }
    catch(std::exception& e)
    {
        std::cout << "Exception caught: " << e.what() << "\n";
    }
    
};
try_and_err();
}();

In [4]:
!bpls Uxt.bp -lavd -n 10

File info:
  of variables:  2
  of attributes: 0
  statistics:    Min / Max 

  double   Uxt   5*{10} = 1.42555e-18 / 1.25756
    (0,0)    0.199179 0.578039 0.900316 1.13446 1.25756 1.25756 1.13446 0.900316 0.578039 0.199179
    (1,0)    1.03021e-05 2.9898e-05 4.65672e-05 5.86781e-05 6.50452e-05 6.50452e-05 5.86781e-05 4.65672e-05 2.9898e-05 1.03021e-05
    (2,0)    5.3286e-10 1.54642e-09 2.40861e-09 3.03502e-09 3.36435e-09 3.36435e-09 3.03502e-09 2.40861e-09 1.54642e-09 5.3286e-10
    (3,0)    2.75612e-14 7.99858e-14 1.24581e-13 1.56981e-13 1.74015e-13 1.74015e-13 1.56981e-13 1.24581e-13 7.99858e-14 2.75612e-14
    (4,0)    1.42555e-18 4.13712e-18 6.44371e-18 8.11955e-18 9.00059e-18 9.00059e-18 8.11955e-18 6.44371e-18 4.13712e-18 1.42555e-18

  double   x     {10} = 0.05 / 0.95
    (0)    0.05 0.15 0.25 0.35 0.45 0.55 0.65 0.75 0.85 0.95



# Now we prepare to read the variables

In [5]:
// int main(int argc, char* argv[] )
[](){
auto try_and_err = []() {
    
//     MPI_Init(NULL, NULL);
//     int rank, size;
//     MPI_Comm_rank(MPI_COMM_WORLD, &rank);
//     MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    try{
        
        //Initialize ADIOS2
        adios2::ADIOS adios(MPI_COMM_SELF, adios2::DebugON);
        adios2::IO io = adios.DeclareIO("CoolingRod1D");
        
        adios2::Engine fr = io.Open("Uxt.bp", adios2::Mode::Read);
        
        std::vector<double> data_x;
        std::vector<std::vector<double>> data_Uxt;
        
        while( fr.BeginStep() == adios2::StepStatus::OK )
        {
            const size_t currentStep = fr.CurrentStep();
            // Inquire some variables
            adios2::Variable<double> var_x = io.InquireVariable<double>("x");
            adios2::Variable<double> var_Uxt = io.InquireVariable<double>("Uxt");
            
            data_Uxt.push_back(std::vector<double>());
            
            //Gets are deferred until EndStep
            if(var_x)
            {
                std::cout << "Reading x in step: " << currentStep << "\n";
                fr.Get(var_x, data_x);
            }
            if(var_Uxt)
            {
                fr.Get(var_Uxt, data_Uxt.back());
                std::cout << "Reading Uxt in step: " << currentStep << "\n";
            }
            
            // data is retrieved
            fr.EndStep();
        }
        fr.Close();
        
        // header
        std::cout << "X ";
        for(auto i =0; i < data_x.size(); ++i)
        {
            std::cout << data_x[i] << "     ";
        }
        std::cout << "\n";
        
        for(auto s=0; s < data_Uxt.size(); ++s)
        {
            std::cout << "U(" << s << ") ";
            for(auto i =0; i < data_x.size(); ++i)
            {   
                std::cout << data_Uxt[s][i] << " "; 
            }
            std::cout << "\n";
        }
    }
    catch(std::exception& e)
    {
        std::cout << "Exception caught: " << e.what() << "\n";
    }
    
    MPI_Finalize();
};
try_and_err();
}();

Reading x in step: 0
Reading Uxt in step: 0
Reading Uxt in step: 1
Reading Uxt in step: 2
Reading Uxt in step: 3
Reading Uxt in step: 4
X 0.05     0.15     0.25     0.35     0.45     0.55     0.65     0.75     0.85     0.95     
U(0) 0.199179 0.578039 0.900316 1.13446 1.25756 1.25756 1.13446 0.900316 0.578039 0.199179 
U(1) 1.03021e-05 2.9898e-05 4.65672e-05 5.86781e-05 6.50452e-05 6.50452e-05 5.86781e-05 4.65672e-05 2.9898e-05 1.03021e-05 
U(2) 5.3286e-10 1.54642e-09 2.40861e-09 3.03502e-09 3.36435e-09 3.36435e-09 3.03502e-09 2.40861e-09 1.54642e-09 5.3286e-10 
U(3) 2.75612e-14 7.99858e-14 1.24581e-13 1.56981e-13 1.74015e-13 1.74015e-13 1.56981e-13 1.24581e-13 7.99858e-14 2.75612e-14 
U(4) 1.42555e-18 4.13712e-18 6.44371e-18 8.11955e-18 9.00059e-18 9.00059e-18 8.11955e-18 6.44371e-18 4.13712e-18 1.42555e-18 
